In [146]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PC
from pinecone import ServerlessSpec
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import FileSystemBlobLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import PyPDFParser
import time

In [147]:
#gerar n versoes da pergunta
#criar novos docs a partir do artigo, categorizados

In [148]:
loader = DirectoryLoader(
    "./rag-projeto/data",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)

In [149]:
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Tamanho máximo de cada pedaço
    chunk_overlap=50 # Sobreposição entre pedaços
)
docs_splitted = text_splitter.split_documents(docs)

texts = [doc.page_content for doc in docs_splitted]


In [150]:

os.environ['PINECONE_API_KEY'] = 'pcsk_6JsENF_EkUKdhWBGJXSsKsbKdU2E5HAuGGq8AvfAvaCjFYj2BJUBxgEPq9GHQYMk2wK6ST'


In [151]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(texts, show_porgress_bar=True)

In [152]:
index_name = "rag"
pc = PC(api_key="pcsk_6JsENF_EkUKdhWBGJXSsKsbKdU2E5HAuGGq8AvfAvaCjFYj2BJUBxgEPq9GHQYMk2wK6ST")
index = pc.Index(index_name)

In [153]:
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


vectorstore = PineconeVectorStore.from_documents(docs_splitted, embedder, index_name=index_name)


In [154]:
query = "Alimentação é mais que ingestão de nutrientes"
similar_docs = vectorstore.similarity_search(query)

In [155]:
similar_docs

[Document(id='98ef6818-2d75-4f07-85bd-7236b2b945ae', metadata={'author': 'Brasil. Ministério da Saúde. Secretaria de Atenção à Saúde. Departamento de Atenção Básica.', 'creationdate': '2014-12-10T15:43:08-02:00', 'creator': 'Adobe InDesign CS6 (Windows)', 'keywords': '1. Programas e Políticas de Nutrição e Alimentação. 2. Alimentação. 3. Preparação de alimentos.', 'moddate': '2015-05-05T10:02:35-03:00', 'page': 23.0, 'page_label': '24', 'producer': 'Adobe PDF Library 10.0.1', 'source': 'rag-projeto/data/guia_alimentar_populacao_brasileira_2ed.pdf', 'title': 'Guia alimentar para a população brasileira', 'total_pages': 158.0, 'trapped': '/False'}, page_content='2322\n     GUIA ALIMENTAR PARA A POPULAÇÃO BRASILEIRA\nOS cINcO PRINcíPIOS \nQUE ORIENTARAM A \nELABORAÇÃO dESTE GUIA\n \nAlimentação é mais que ingestão de nutrientes\nalimentação diz respeito à ingestão de nutrientes, como também aos alimentos \nque contêm e fornecem os nutrientes, a como alimentos são combinados entre si e \npr

In [156]:
import numpy as np

# Vamos supor que embeddings é uma lista de vetores
# Pegamos o primeiro vetor para a busca
query = "Alimentação é mais que ingestão de nutrientes"
query_vector = model.encode(query)

# Fazendo a busca no Pinecone
response = index.query(
    vector=query_vector.tolist(),  # transforma em lista pura
    top_k=50,
    include_values=True  # Garantir que estamos incluindo os valores (documentos)
)
print(response)
# Ordenando os resultados pelo score de forma decrescente
sorted_matches = sorted(response['matches'], key=lambda x: x['score'], reverse=True)

# Mostrando os resultados ordenados pelo score e o conteúdo (valores)
for match in sorted_matches:
    print(f"ID: {match['id']}, Score: {match['score']}")
    
    # Exibindo o conteúdo do documento (valor associado ao ID)
    print(f"Conteúdo: {match['values']}")  # Aqui, 'values' contém o conteúdo do documento


{'matches': [{'id': '98ef6818-2d75-4f07-85bd-7236b2b945ae',
              'score': 0.829392731,
              'values': [0.0378739722,
                         -0.00204937137,
                         -0.0595914759,
                         -0.0320412628,
                         -0.0637483522,
                         -0.00750200637,
                         0.0535111316,
                         0.0461575203,
                         -0.0160649214,
                         0.00631010719,
                         0.05612633,
                         -0.0324049,
                         -0.00143276725,
                         -0.0577299148,
                         -0.0596327521,
                         -0.0881166086,
                         0.0152291488,
                         0.026485784,
                         -0.060532961,
                         0.000778467394,
                         0.0854785517,
                         -0.0510316789,
                         -0.068969